<a href="https://colab.research.google.com/github/capozzolicaluan/MSWEPtest/blob/main/ajuste_bases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Limpar todas as variáveis
from IPython import get_ipython
get_ipython().magic('reset -sf')

# Código para reiniciar o ambiente do Colab
import os
os.kill(os.getpid(), 9)

In [18]:
# Importando as bibliotecas necessárias
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from datetime import datetime, timedelta
import gc
import os as os
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import seaborn as sns
from google.colab import data_table
import math
from matplotlib.colors import TABLEAU_COLORS, same_color
import matplotlib.pyplot as plt
import glob
import xarray as xr
from tqdm import tqdm # para monitorar o progresso do processo


# Liberar memória manualmente
_ = gc.collect()

In [19]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [91]:
caminho_dados_mswep = "/content/drive/MyDrive/Colab/txts"
caminho_dados_hidro = "/content/drive/MyDrive/"

In [21]:
ini_analise_dataBR = "01/01/2021 00:00:00" # data e hora de inicio horas sempre de 15 em 15 min
fim_analise_dadaBR = "31/12/2024 23:45:00" # data e hora de  horas sempre de 15 em 15 min

In [22]:
start_datetime = pd.to_datetime(ini_analise_dataBR, format="%d/%m/%Y %H:%M:%S")
end_datetime = pd.to_datetime(fim_analise_dadaBR, format="%d/%m/%Y %H:%M:%S")

In [39]:
# Converter as strings de data e hora para objetos datetime do pandas
start_datetime = pd.to_datetime(ini_analise_dataBR, format="%d/%m/%Y %H:%M:%S")
end_datetime = pd.to_datetime(fim_analise_dadaBR, format="%d/%m/%Y %H:%M:%S")

# Gerar a sequência de datas a cada 3 horas
datetime_sequence = pd.date_range(start=start_datetime, end=end_datetime, freq='3h')

# Criar um DataFrame para armazenar os resultados
df_dates = pd.DataFrame(datetime_sequence, columns=['datetime'])

# Extrair dia juliano, ano e hora
df_dates['julian_day'] = df_dates['datetime'].dt.dayofyear
df_dates['year'] = df_dates['datetime'].dt.year
df_dates['hour'] = df_dates['datetime'].dt.hour

# Crie a lista de strings no formato YYYYJJJ.HH.txt
file_list = [f"{row['year']}{row['julian_day']:03d}.{row['hour']:02d}.txt" for index, row in df_dates.iterrows()]

In [40]:
output_data = []

for file_name in tqdm(file_list, desc="Processing files"):
  file_path = os.path.join("/content/drive/MyDrive/Colab/txts/", file_name)
  # Read the text file
  df = pd.read_csv(file_path, header=0, sep=',')

  # Select the "chuva" column
  chuva_column = df['chuva']

  # Transpose the "chuva" column
  chuva_row = chuva_column.values.tolist()

  # Save the transposed column to the output variable
  output_data.append(chuva_row)

# Create the output DataFrame
output_df = pd.DataFrame(output_data)

# column names
output_df.columns = df['nome'].astype(int).tolist()

Processing files: 100%|██████████| 11688/11688 [1:03:26<00:00,  3.07it/s]


In [253]:
# Definir o caminho para salvar o arquivo TXT
output_file_path = "/content/drive/MyDrive/Colab/txts/matriz_mswep.txt"

# adicionar a coluna de data-hora
# output_df.insert(0, 'datetime', df_dates['datetime'])

# Salvar o DataFrame como um arquivo TXT
output_df.to_csv(output_file_path, sep=';', index=False, header=True)

print(f"DataFrame salvo em: {output_file_path}")

DataFrame salvo em: /content/drive/MyDrive/Colab/txts/matriz_mswep.txt


In [ ]:
# Especificar o caminho do arquivo Excel
caminho_excel = "/content/drive/MyDrive/dados_plu_2020-01-01_2024-12-31.xlsx"

# Ler todas as abas do arquivo Excel em um dicionário de DataFrames
# A chave do dicionário será o nome da aba
dict_dfs = pd.read_excel(caminho_excel, sheet_name=None)

# Criar uma lista de DataFrames a partir dos valores do dicionário
lista_dfs_excel = list(dict_dfs.values())

In [247]:
output_hidro = [None]*len(dict_dfs)

for indice in range(len(dict_dfs)):
  dados = lista_dfs_excel[indice]
  dados_ = dados.iloc[35136:,[1,4]]
  dados_['DataHora'] = pd.to_datetime(dados_['DataHora'])
  dados_ = dados_.set_index('DataHora')
  dados_3h = dados_['Chuva'].resample('3H', label='right', closed='right').sum(min_count=10)
  output_hidro[indice] = dados_3h.values.tolist()

output_hidro_df = pd.DataFrame(output_hidro).T

In [248]:
output_hidro_df.columns = df['nome'].astype(int).tolist()
output_hidro_df = output_hidro_df.iloc[:-1]

In [249]:
# adicionar a coluna de data-hora
output_hidro_df.insert(0, 'datetime', df_dates['datetime'])

In [256]:
# Salvar o DataFrame como um arquivo TXT
# Definir o caminho para salvar o arquivo TXT
output_file_path = "/content/drive/MyDrive/Colab/txts/matriz_hidro.txt"

output_hidro_df.to_csv(output_file_path, sep=';', index=False, header=True)